In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [2]:
tf.random.set_seed(777)

In [3]:
rootPath = './vision/'

In [4]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [5]:
imageGenerator = ImageDataGenerator(
    rotation_range=90, # 회전각도 
    horizontal_flip=True,  # 이미지 뒤집기 
    preprocessing_function = get_random_eraser(v_l=0, v_h=255), # 전체 부분의 일부를 가림 
    validation_split=.2
)

In [6]:
trainGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(100, 100),
    subset='training'
)

Found 1028 images belonging to 3 classes.


In [7]:
validationGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(100, 100),
    subset='validation'
)

Found 256 images belonging to 3 classes.


In [8]:
model = Sequential()
model.add(ResNet50(include_top=True, weights=None, input_shape=(100, 100, 3), classes=3))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 3)                 23593859  
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_________________________________________________________________


In [9]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['acc'],
)

In [10]:
epochs = 50
history = model.fit_generator(
    trainGen, 
    epochs=epochs,
    steps_per_epoch=trainGen.samples / epochs, 
    validation_data=validationGen,
    validation_steps=trainGen.samples / epochs,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
21/20 [==============================] - ETA: 0s - loss: 1.2214 - acc: 0.6134WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000002ED5FD7E730> and will run it as-is.
Please report this to the TensorFlow team. When filing the bu

In [11]:
testGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'test_set'),
    target_size=(100, 100),
)

Found 15 images belonging to 3 classes.


In [12]:
pred = model.predict(testGen)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [13]:
img_list = testGen.filenames

In [14]:
img_list

['burn_disease\\burn1.png',
 'burn_disease\\burn2.png',
 'burn_disease\\burn3.png',
 'burn_disease\\burn4.png',
 'burn_disease\\burn5.png',
 'healthy\\hel1.jpg',
 'healthy\\hel2.jpg',
 'healthy\\hel3.jpg',
 'healthy\\hel4.jpg',
 'healthy\\hel5.jpg',
 'leafspot\\leafspot1.png',
 'leafspot\\leafspot2.png',
 'leafspot\\leafspot3.png',
 'leafspot\\leafspot4.png',
 'leafspot\\leafspot5.png']

In [15]:
pred_list = []
for i in range(len(pred)):
    pred_list.append(np.argmax(pred[i]))

In [16]:
df = pd.DataFrame(list(zip(img_list, pred_list)), columns =['Name', 'pred'])

In [17]:
df.to_csv('submission.csv', index=False)

In [18]:
print("정확도 :", model.evaluate_generator(testGen)[1])

Instructions for updating:
Please use Model.evaluate, which supports generators.
정확도 : 0.9333333373069763
